In [47]:
# Import all relevant packages here.
# We need folium for the maps and pandas for the dataframes
import folium
from folium.plugins import HeatMap
import pandas as pd

In [55]:
# Import our test data
birds = pd.read_csv("Birds_test.csv")

# Drop all duplicates for latitude and longitude
long_lat = birds[['Latitude', 'Longitude']].drop_duplicates()

# Create a list of unique birds for each unique longitude
# Therefore common birds such as blackbirds etc. won't be counted twice in the heatmap
bird_list = birds.groupby('Longitude').agg({'Bird': 'unique'}).reset_index()

# Merge the unique bird list back onto the unique longitudes and latitudes
long_lat = pd.merge(long_lat, bird_list, on = 'Longitude')

In [56]:
# Start a map with the location around the UK.
# I've currently set zoom to 7 due to where the current data is located
birds_hmap = folium.Map(location = [51, -3], zoom_start = 7)

# Plotting heat map values using the longitude and latitude values from the original dataframe
# The more birds at one location means the more intense the colour
heatmap_values = HeatMap(list(zip(birds.Latitude.values, birds.Longitude.values)),
                  min_opacity = 0.2,
                  radius = 25,
                  blur = 20, 
                  max_zoom = 1)

# Add the heatmap values to the Map
birds_hmap.add_child(heatmap_values)


In [57]:
# For each row in the unique longitudes we have above we are going to plot a marker
# The marker will be the list of unique birds created above
# Add the markers to the heatmap above
for row in long_lat.iterrows():
    row_values = row[1]
    location = [row_values['Latitude'], row_values['Longitude']]
    popup = (str(row_values['Bird'])).replace("[", "").replace("]", "")
    marker = folium.Marker(location = location, popup = popup)
    marker.add_to(birds_hmap)

In [58]:
# Display the heat map to see what we hav
display(birds_hmap)